## Create a filtered/cleaned dataset for accidents between 2005 and 2020

### Introduction

In this Notebook we will define the filtering rules for our dataset.
The generated dataset will contain informations about bicycle accidents in France form 2005 to 2020.

***

### Import libraries & packages

In [65]:
import pandas as pd
import numpy as np

### Import Merged dataset

In [66]:
#import merged dataset for 2005..2020:
df= pd.read_csv('data/merged_data_2005_2020.csv', sep=';', index_col=0)

#don't worry about the warning, all the concerned columns will be dropped in the next steps

C:\Users\WonderSSJ9\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (9,22,24,25,26,28,38,39) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\WonderSSJ9\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [67]:
#we select bicycles category ; for the cars replace with 7; for motocycles replace with [2,30,31,32,33,34]
df= df[df.catv==1]

#display merged dataset columns:
print("Columns of the 'raw' dataset before filtering : \n\n", list(df.columns))

Columns of the 'raw' dataset before filtering : 

 ['Num_Acc', 'place', 'catu', 'grav', 'sexe', 'trajet', 'secu1', 'locp', 'actp', 'etatp', 'an_nais', 'num_veh', 'an', 'mois', 'jour', 'hrmn', 'lum', 'agg', 'int', 'atm', 'col', 'com', 'adr', 'lat', 'long', 'dep', 'catr', 'voie', 'v1', 'v2', 'circ', 'nbv', 'pr', 'pr1', 'vosp', 'prof', 'plan', 'lartpc', 'larrout', 'surf', 'infra', 'situ', 'senc', 'catv', 'occutc', 'obs', 'obsm', 'choc', 'manv', 'motor']


***

### Changes to apply on the dataset

- drop : 'dep', 'com', 'col', 'adr', 'lat', 'long' ; 'senc', 'obs', 'obsm', 'choc', 'manv', 'motor', 'occutc' ; 'voie', 'v1', 'v2', 'pr', 'pr1', 'lartpc', 'larrout', 'plan' ; 'place', 'locp', 'actp', 'etatp' ; 'catv'
- drop 'num_veh' & 'Numm_Acc' for the prediction model
- replace 0 in 'surf' with mode value: 1
- replace -1 values in 'surf' & 'atm' with the mode value: 1
- compare 'surf' and 'atm' : to see if we keep them both
- make a count_values for 'prof': to see if it should be removed
- compare 'situ' and 'vosp': to see if we keep them both
- make a count_values for 'catu':  to see if it should be removed
- analyse 'trajet' and see if it has an influence on severity of accidents 'grav'
- change the labeling in 'secu1' so that data before 2019 matches data after
- rename 'secu1' to 'secu'
- analyse 'secu'
- replace 'an_nais' with 'age'
- limit 'infra' to two categories
- limit 'nbv' to 4 categories

- **Drop unnecessary + Ids columns:**

In [68]:
df.drop( ['dep', 'com', 'col', 'adr', 'lat', 'long', 'senc', 'obs', 'obsm', 'choc', 'manv', 'motor', 'occutc', 'voie', 'v1',
 'v2', 'pr', 'pr1', 'lartpc', 'larrout', 'place', 'locp', 'actp', 'etatp', 'num_veh', 'Num_Acc', 'catv'] , axis=1, inplace=True)

- **Comparing between 'surf' & 'atm' :**

In [69]:
#first we replace -1 & 0 values in surf and atm with mode (1) 
df[['surf','atm']] = df[['surf','atm']].replace([-1,0], 1)

#compare 'surf' & 'atm'
pd.crosstab(df['surf'],df['atm'], normalize=True)

atm,1,2,3,4,5,6,7,8,9
surf,,,,,,,,,
1,0.838578,0.006420,0.000739,0.000170,0.001756,0.001708,0.018217,0.017745,0.001696
2,0.021802,0.059266,0.010647,0.000703,0.000703,0.000388,0.000545,0.009278,0.000896
3,0.000230,0.000133,0.000230,0.000012,0.000000,0.000000,0.000012,0.000048,0.000012
4,0.000036,0.000012,0.000048,0.000000,0.000000,0.000000,0.000000,0.000000,0.000012
5,0.000351,0.000000,0.000000,0.000557,0.000000,0.000000,0.000012,0.000000,0.000012
6,0.000121,0.000012,0.000012,0.000000,0.000000,0.000000,0.000000,0.000036,0.000000
7,0.000557,0.000012,0.000000,0.000121,0.000024,0.000000,0.000048,0.000157,0.000533
8,0.000703,0.000061,0.000000,0.000000,0.000012,0.000000,0.000000,0.000000,0.000000
9,0.004094,0.000073,0.000000,0.000012,0.000036,0.000000,0.000097,0.000121,0.000182


Based on the crosstable, there's a strong dependency between category 1 of each feature, but for the other categories it's not clear.
So wee keep both of the features.

- **Analysing 'prof':**

In [70]:
#prercentage of each value:
print(df.prof.value_counts(normalize=True))

 1    0.773304
 2    0.144089
 0    0.055438
 4    0.013857
 3    0.010744
-1    0.002568
Name: prof, dtype: float64


Based on the results, there are 6.2% of unspecified values. We drop feature.

In [71]:
#drop 'prof'
df.drop('prof', axis=1, inplace=True )

- **Comparing 'situ' & 'vosp:**

In [72]:
#prercentage of each value:
print(df.vosp.value_counts(normalize=True),'\n')
print(df.situ.value_counts(normalize=True),'\n')

 0    0.798680
 1    0.109872
 3    0.043362
 2    0.042563
-1    0.005523
Name: vosp, dtype: float64 

 1    0.811010
 5    0.072892
 0    0.046730
 3    0.030220
 4    0.025182
-1    0.006504
 6    0.002919
 2    0.002774
 8    0.001768
Name: situ, dtype: float64 



Since percentage of each cat 1 (*présence piste cyclable*) in vosp is diffrent from cat 5 in situ ( *accident sur piste cyclable*), we keep the two.

- **Analysing 'catu':**

In [73]:
df.catu.value_counts(normalize=True)

1    0.933660
3    0.054239
2    0.010344
4    0.001756
Name: catu, dtype: float64

We drop 'catu'=4, because it correponds to 'electric scooter' (the catv 99 didn't exist before 2019). We keep 'catu'=1 because our goal is to predict accidents for bicycle drivers.

In [74]:
#keep rows with 'catu'=1
df= df[df.catu==1]

#drop 'catu'
df.drop('catu', 1, inplace=True)

- **Analysing 'trajet'**

In [75]:
df.trajet.value_counts(normalize=True)

 5    0.493987
 0    0.205947
 1    0.150824
 9    0.060675
 2    0.039555
 4    0.029877
 3    0.018538
-1    0.000597
Name: trajet, dtype: float64

0 and -1 corresponds to unspecified values, wich represents more than 20%. We drop 'trajet'.

In [76]:
#drop 'trajet'
df.drop('trajet', axis=1, inplace=True )

- **Replace 'an_nais' with 'age':**

In [77]:
#we drop unspecified lines for 'an_naiss':
df= df[df.an_nais!=-1]

#we calculate 'age' and we drop 'an_nai':(we add 1 to avoide the 0 cases)
df['age'] = df.an - df.an_nais + 1
df.drop('an_nais', axis=1, inplace=True )

#select age<100
df= df[df.age<100]

- **Adjust data labeling for 'secu':**

In [78]:
#Rename secu1 to secu
df.rename({'secu1':'secu'}, axis = 'columns', inplace= True)

#Adjust values in 2019-2020 to match the labeling in =<2018
df.secu.replace([[5,6,7], 8], [9, -1], inplace=True )

#counting values
df.secu.value_counts(normalize=True)

-1    0.496074
 2    0.250672
 0    0.198261
 9    0.030748
 4    0.023610
 1    0.000493
 3    0.000143
Name: secu, dtype: float64

Most of the 'secu' values for bicycle are not specified, therefore we drop it

In [79]:
#drop 'secu'
df.drop('secu', axis=1, inplace=True )

- **Limiting categories in 'infra' to two: *Yes*, *No***

In [80]:
#transform categories in 'infra' to 1 (presence of roadworks) & 0 (absence of roadworks)
df.loc[df.infra >= 1, 'infra' ] = 1

#count values
df.infra.value_counts(normalize=True)

 0    0.869362
 1    0.116088
-1    0.014550
Name: infra, dtype: float64

In [81]:
df.nbv.value_counts()

 2     44354
 1     13139
 0      8176
 4      5001
 3      4531
 5       640
 6       597
-1       331
 8       116
 7        78
 10       22
 20       21
 9        13
 50       12
 30        4
 12        3
 13        2
 24        1
 26        1
 90        1
 28        1
 11        1
Name: nbv, dtype: int64

- **Limiting categories in 'nbv' to two:**
    
    1. We replace 0 value with mode of nbv: it's impossible to have 0 lanes on a road, so we assume that in this case it was the mode, wich is 2 lanes.
    2. We include the lanes with 4 lanes or more in the 4th category.

In [82]:
#replace 0 with mode:
df['nbv'] = df.nbv.replace(0,2)

#We create a category for more than 4 lanes:
df.loc[df.nbv>=4 , 'nbv' ] = 4

- **Analysing the unspecified values:**

In [83]:
#calculate unspecified value of each column
unspec_val= {}
for col in df.columns:
    c = df[df[col]==-1 ][col].count()
    unspec_val[col]= round(c / df.shape[0]*100 , 2)

unspec_val

{'grav': 0.0,
 'sexe': 0.0,
 'an': 0.0,
 'mois': 0.0,
 'jour': 0.0,
 'hrmn': 0.0,
 'lum': 0.0,
 'agg': 0.0,
 'int': 0.0,
 'atm': 0.0,
 'catr': 0.0,
 'circ': 1.03,
 'nbv': 0.43,
 'vosp': 0.56,
 'plan': 0.29,
 'surf': 0.0,
 'infra': 1.45,
 'situ': 0.69,
 'age': 0.0}

The remaining unspecified values have a small percentage. We can drop them.

In [84]:
#drop -1 rows in df:
for col in df.columns.tolist():
    if df[col].dtype=='int64':
        df = df[df[col] >= 0]

#if we replace them with nan and then drop nan, types change to float

- **Reseting index column:**

In [85]:
#is useful if you are going to set index argument to True in '.t_csv'
df.reset_index(drop=True, inplace=True)


***
### Save the filtered dataset:
- Export the filtered file for bicycle accidents to *bicycle_accidents_05_20_filtered.csv'* in **data/** folder:

In [86]:
#Generate the filtered dataset for bicycles:
df.to_csv(path_or_buf= 'data/bicycle_accidents_05_20_filtered.csv',sep=';', index=False)

In [87]:
#test
y= pd.read_csv('data/bicycle_accidents_05_20_filtered.csv', sep=';' )
print("Columns of the filtered dataset : ", y.columns.tolist())
y

Columns of the filtered dataset :  ['grav', 'sexe', 'an', 'mois', 'jour', 'hrmn', 'lum', 'agg', 'int', 'atm', 'catr', 'circ', 'nbv', 'vosp', 'plan', 'surf', 'infra', 'situ', 'age']


,grav,sexe,an,mois,jour,hrmn,lum,agg,int,atm,catr,circ,nbv,vosp,plan,surf,infra,situ,age
0,4,1,2005,1,13,19:45,5,2,1,1,3,2,2,0,1,1,0,1,59
1,3,1,2005,1,19,10:45,1,1,1,7,3,2,2,0,3,1,0,1,21
2,4,1,2005,1,26,13:15,1,1,9,1,3,2,2,0,1,1,0,1,72
3,3,2,2005,1,3,13:30,1,2,1,1,3,2,2,0,2,1,0,1,52
4,4,1,2005,1,29,18:30,3,1,1,1,3,2,2,0,1,1,0,1,75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74922,3,2,2020,11,30,14:25,1,2,1,1,7,2,4,1,1,1,0,5,61
74923,4,2,2020,11,30,17:38,1,2,1,1,4,2,2,0,1,1,0,1,67
74924,4,1,2020,12,1,11:11,1,2,3,1,4,2,2,0,1,1,0,1,67
74925,4,2,2020,12,1,13:30,1,2,4,2,4,2,2,0,3,2,0,8,61
